# Python and performance

In this post we discuss various aspects of performance when programming in Python.

## Compiled vs interpreted language implementations

Compiled implementations of languages require the use of a *compiler*. This is a computer program which converts the human-readable *source file* written by a human into machine-readable code. The output is an *executable file*, which is a sequence of instructions which the central processing unit (CPU) of the computer is able to process and execute directly. Modern CPUs can execute billions of these elementary instructions every second. Because the instructions can directly be processed by the CPU, the executable is *architecture-dependent*. In other words, an executable compiled to run on your laptop will probably not run on your phone. The most common architectures today are x86_64 for computers and ARM for mobile and embedded devices.

C++ is an ubiquitous compiled programming language; it was created to add object-oriented programming functionalities to C, and is designed for performance over all else. Let's look at a simple sample C++ file, and its translation into x86_64 code.

In [ ]:
!cat example0.cpp

This program defines three variables `a`, `b` and `c`, and stores the sum of `a` and `b` into `c`. It then displays the result. To compile it, we'll use `g++`, an open source C++ compiler.

In [ ]:
!g++ example0.cpp -O0 -o example0

Let's run the compiled executable:

In [ ]:
!./example0

A *disassembler* decomposes a binary executable into individual instructions for humans to read (this may look like a simple task, but is not an exact science!). Let's disassemble the executable we just created.

In [ ]:
!objdump -d example0

There is a lot there, but we are only interested in the section starting with `<main>:` since it directly corresponds to the C++ code we wrote. Let's isolate that part.

In [ ]:
!objdump -d example0 | sed -n '/<main>:/,/retq/p'

It's possible to recognise what we wrote in C++ after a while... for example, lines 908, 90f and 916 define the three variables, lines 91d and 920 push the contents of the variables to the CPU registers, and finally line 923 adds the value of the two registers! For more complicated programmes, this becomes tricky to follow, to say the least... but this is also the point: this code is only supposed to be read by a machine. However, we do see that the critical part of our programme was converted to a handful of CPU instructions, which is great for performance. If you'd like to get more hands-on with assembly code, here are three useful resources:
* https://godbolt.org/: generates assembly code on the fly and displays it next to your C++ code. Very useful to learn assembly!
* https://www.agner.org/optimize: contains, amongst other things, a guide on optimization for x86 processors (AMD, VIA and Intel). If you ever wondered what `mov`, `1ea` or `callq` do, what their throughput and latencies are on a given CPU, this is the place to go.
* Intel publishes "intrinsics" i.e. C functions that map directly to individual CPU instructions. More here: https://software.intel.com/sites/landingpage/IntrinsicsGuide/#

Exercise: replace `-O0` by `-O3` in the cell `!g++ example0.cpp -O0 -o example0`, and re-run the cell. Reexamine the executable. What happened?

Interpreted language implementations function differently from compiled ones. Instead of being converted to CPU instructions directly, the source code is parsed then intermediate, machine-independent *opcode* (or *bytecode*) is generated. This intermediate code is then *interpreted* by an interpreter (for the Python language, the reference interpreter is CPython - so called because it is written in C - and is the program run when you call `python` on the command line). In a way, the interpreter is a "machine within the machine": it takes the place of the CPU (though of course it itself is being run by the CPU!). This explains some of the performance loss incurred when running Python code, since the presence of an interpreter induces overhead. We've written a Python program equivalent to the C++ one above:

In [ ]:
!cat example1.py

In [ ]:
!python ./example1.py

One can explicitly ask the Python executable to generate the intermediate opcode. Ever wondered what the `.pyc` files present in `__pycache__` were?

In [ ]:
!python -m compileall example1.py

In [ ]:
!python view_bytecode.py __pycache__/example1.cpython-36.pyc

Each group of opcodes directly corresponds to each line in `example1.py` (and this is much more readable than assembly code...). To execute each opcode, many CPU instructions are required - but how many? To get a feel for this, let's inspect the source code of the CPython interpreter! In fact, let's just consider the crucial opcode, which is `BINARY_ADD` (adds two numbers together).

In [ ]:
# Uncomment the next line if you want to inspect more of the CPYthon source code than what is looked at in this notebook.
# !git clone https://github.com/python/cpython.git

Feel free to have a closer look at `ceval.c`. We'll only show the relevant parts, since it is a lot of C code to take in in one go. Effectively, you'll find a big `switch` statement, followed by a lot of `case` statements, each corresponding to one opcode:

In [ ]:
!grep 'case TARGET' cpython/Python/ceval.c | tail -20

Although this is C code, it's not hard to see what this is doing: the programme is looping over each opcode, and based on the value of the opcode does someting different. So what happens when `BINARY_ADD` is encountered?...

In [ ]:
!sed -n '/BINARY_ADD/,/DISPATCH()/p' cpython/Python/ceval.c

Looks fairly complicated for a simple sum... but really all this does is call PyNumber_Add provided the arguments are not two Unicode strings (in which case they will be concatenated). So let's have a look at `PyNumber_Add`...

In [ ]:
!sed -n '/PyNumber_Add/,/return result/p' cpython/Objects/abstract.c

So now we call `binary_op1`. If the result is `Py_NotImplemented` we do... something, else we return the result. Fine, let's have a look at `binary_op1`.

In [ ]:
!sed -n '/binary_op1/,/Py_RETURN_NOTIMPLEMENTED/p' cpython/Objects/abstract.c

This seems to be doing some type checking. If the object in question a "number type" (if so define the `slotv` function...) and more checking, and finally if possible run `slotv`. And do make sure to check that the returned type is not `PyNotImplemented`, of course...

What's going on here is that Python is (very) dynamically typed: types are associated to *values* rather than *variables*. So whenever an operation is executed on an object, the interpreter has to check that the relevant object supports it. And take care of potential failures. Everything is an object in Python! What was one CPU instruction in our C++ example is quickly turning into many more when the Python interpreter is running the show. Because of the flexibility that Python offers (duck typing, etc), a lot of optimisations that are available to other interpreted languages are simply not available to it. That is the main reason why Python is so slow! It's common for a C++ implementation of a function to be 100x faster than a Python implementation.

If you'd like to explore the CPython source code in more details, why not use `gdb` (the GNU debugger) and step through the execution of the CPython interpreter? You can find a tutorial here: https://medium.com/@skabbass1/how-to-step-through-the-cpython-interpreter-2337da8a47ba

## Improving performance

There are a number of ways in which you can improve the performance of your Python code. Normally, your programmes will spend most of their time in the same portion of the code - we call these *hotspots*. It is these hotspots that really need to be sped up, and not the rest of your code. For example, if your programme consists of reading a file from disk, parsing it into a custom data structure, and then running some bespoke ML algorithm on that structure, then probably only the latter step needs optimisation. We will now discuss three ways of speeding up your Python code: Cython, Numba and writing custom extensions. We won't discuss some others like using a different Python interpreter such as PyPy or Jython.

### Cython

Cython is an *optimising static compiler* for the Python language and the *extended Cython language, which is a superset of the Python programming language*. What does that mean? Cython compiles Python code, instead of interpreting it. Second, it defines a set of annotations that you may use on your Python code. These annotations are not part of the Python language. Rather, they help Cython understand the types of your variables and the signatures of your functions. Really, they let you write C code with Python syntax. The corollary here is that to use Cython well, you need to understand C. In particular, Cython is most succesful at optimising your code when you do not use Python's convenient features like dynamically typed variables, introspection, and so on. Let's give Cython a go. To compare its performance with pure Python, we'll test a function adding the first $10^8$ integers and returning the output.

In [ ]:
!cat add_integers_python.py

Cython code should be written in files ending in `.pyx`.

In [ ]:
!cat add_integers_cython.pyx

This is fairly similar to the pure Python function. The main difference is that we annoted the type of the variables using the `cdef` keyword, and also annotated the function arguments. Note that `uint64_t` stands for unsigned 64 bit integer. It can hold any non-negative integer between 0 and $2^{64} - 1$, inclusive. To compile `add_integers_cython.pyx` into a Python extension, we have written a custom `setup.py` file:

In [ ]:
!cat setup.py

We build the extension as follows:

In [ ]:
!python setup.py build_ext --inplace

Let's compare the pure python implementation to the Cython one.

In [ ]:
import time

In [ ]:
def timefunc(func, *args, **kwargs):
    t0 = time.time()
    func(*args, **kwargs)
    t1 = time.time()
    return t1 - t0

In [ ]:
from add_integers_python import f

In [ ]:
n = 100_000_000

In [ ]:
timefunc(f, n)

In [ ]:
from add_integers_cython import f as f_cython
from add_integers_cython import f_no_annotations as f_cython_no_annotations

In [ ]:
timefunc(f_cython_no_annotations, n)

In [ ]:
timefunc(f_cython, n)

That's a roughly 200x improvement when using Cython annotations! When not annotating the code, Cython still provides a 2X performance improvement (recall that the function is statically compiled rather than interpreted). You may want to check out `add_integers_cython.c`. The `add_integers_cython.pyx` file was tranlated into this C source file, and the latter was then compiled using a C compiler. For more information on Cython, see https://cython.org/

### Numba

Numba is easier to use than Cython. Numba's premise is that rather than having you specify types manually, it will try to infer them at runtime for you. As a consequence, most of the time all you have to do is to decorate your functions to tell numba to try to attempt and optimise them. No need for a different language here.

In [ ]:
from numba import jit

In [ ]:
f_numba = jit(nopython=True)(f)

In [ ]:
f_numba(10)

In [ ]:
timefunc(f_numba, n)

In [ ]:
from add_integers_numba import f_numba, f_numba_with_signature

In [ ]:
timefunc(f_numba_with_signature, n)

In [ ]:
f_numba_with_signature(n)

Woah! That was fast. How is this possible? You may want to use the `inspect_asm()` or `inspect_llvm()` methods of `f_numba` to check your theory.

It is also possible, and sometimes necessary, to specify the signatures of your functions directly, rather than to let `numba` find out what they are on its own. That's how you do it:

In [ ]:
from numba import uint64

In [ ]:
f_numba_with_signature = jit(uint64(uint64), nopython=True)(f)

In [ ]:
f_numba_with_signature(100)

Numba has (very basic) support for classes, too.

In [ ]:
from numba import jitclass
from collections import OrderedDict

In [ ]:
@jitclass(OrderedDict([("x", uint64), ("y", uint64)]))
class Point2D:
    def __init__(self, x, y):
        self.x = x
        self.y = y

Some trickery is required if you want to specify the signature of functions using your jitclassed classes...

In [ ]:
NumbaPoint2DType = Point2D.class_type.instance_type

@jit(NumbaPoint2DType(NumbaPoint2DType, NumbaPoint2DType), nopython=True)
def add_points(a, b):
    return Point2D(a.x + b.x, a.y + b.y)

In [ ]:
result = add_points(Point2D(1, 2), Point2D(3, 4))

In [ ]:
(result.x, result.y)

Class support is limited, however. For example, it is not possible for a class to reference itself - something you would do using pointers in C++. So defining recursive structures is very difficult (I have seen one example online, and could not adapt it to my needs). Again, knowing some C is useful here to understand what can and cannot be done with `numba`. For more information, follow the documentation here: http://numba.pydata.org/

### Using C/C++ directly

If C knowledge is effectively required to properly use Cython or Numba, might we not want to code our functions directly in C? Indeed, this is possible, and is probably preferable. One of the risks in using Cython or Numba is that they may not always suit your needs (see limitations above). You would not want to realise this in the middle of a project...

#### The C Foreign Function Interface (`cffi`) package

The `cffi` package allows you to use compiled C libraries directly. Here is our favourite function coded up in C:

In [ ]:
!cat add_integers_c.h

In [ ]:
!cat add_integers_c.c

Let us compile it into a *shared library*. A shared library is a file containing machine code intended to be used by executable files. Shared libraries can be *statically linked*, meaning that their code is directly integrated into a single monolithic executable right after compilation, or can be *dynamically linked*, meaning that they are made available to executables at runtime. On Linux, static library names end in `.a` and dynamic libraries names end in `.so`. On Windows they end in `.lib` and `.dll`, respectively, and on OS X in `.a` and `.dylib`, respectively. It's often better to dynamically link your libraries. This makes it possible for only one instance of the shared code to be loaded into system memory, but most of all for the library code to be modified independently from the executable code (very useful when a bug is discovered!).  

In [ ]:
!gcc -c -Wall -Werror -fPIC add_integers_c.c

In [ ]:
!gcc -shared -Wl,-soname,libadd_integers_c.so -o libadd_integers_c.so add_integers_c.o

Now we can use `cffi` to make functions from this shared library available from a Python module.

In [ ]:
from cffi import FFI

ffibuilder = FFI()

# cdef() expects a single string declaring the C types, functions and
# globals needed to use the shared object. It must be in valid C syntax.
ffibuilder.cdef(
    """
    uint64_t f(uint64_t);
"""
)

# set_source() gives the name of the python extension module to
# produce, and some C source code as a string.  This C code needs
# to make the declarated functions, types and globals available,
# so it is often just the "#include".
ffibuilder.set_source(
    "_add_integers_cffi",
    """
     #include "add_integers_c.h"   // the C header of the library
""",
    libraries=["add_integers_c"],
    extra_link_args=[
        "-L/project/performance/examples"
    ],  # Must add this to tell the linker where to find our shared library
)  # library name, for the linker

ffibuilder.compile(verbose=True)

In [ ]:
from _add_integers_cffi import lib

In [ ]:
lib.f(100)

#### Writing a C++ extension with Boost

Boost (https://www.boost.org) has a library which wraps all the boiler plate code required to write Python extensions manually (it will take care of reference counts for you, for example...). The result is that it is quite simple to expose your C++ classes as Python classes. Installation is a bit tricky, though. For reference, a list of instructions follows. However, we have included a script to automate this process. To run this script, close this notebook, open a new terminal and run
```
cd install-boost && ./install_boost.sh
```
This will restart Jupyter.

Manual installation instructions:

1. make sure that `numpy` is installed.  For example using `pip`: 

   ```
   pip install numpy
   ```

2. Download the Boost C++ libraries version 1.69 available here:
   https://dl.bintray.com/boostorg/release/1.69.0/source/boost_1_69_0.tar.gz for
   example by running
    
   ```
   cd /tmp && wget https://dl.bintray.com/boostorg/release/1.69.0/source/boost_1_69_0.tar.gz
   ```

3. Extract the Boost C++ libraries:

   ```
   cd /tmp && tar -xvf boost_1_69_0.tar.gz
   ``` 
   
   This creates a directory `boost_1_69_0`.
   Define an environment variable named `BOOST_ROOT`
   pointing to the root of your newly installed Boost distribution:
   
   ```
   export BOOST_ROOT=/tmp/boost_1_69_0
   ```

4. Compile the Boost Python and Numpy shared libraries. Run 

	```
	cd $BOOST_ROOT && ./bootstrap.sh
	```
	
   This will create a `bjam` configuration file named `project-config.jam`.
   Bjam (the Boost build tool) is not always capable of detecting the correct
   Python include paths, so we'll need to fix `project-config.jam` manually.
   Look for a line resembling 
   ```
   using python : 3.6 : /opt/anaconda/envs/Python3 ;
   ```
   in this file, and replace it with
    
   ```
   using python : 3.6 : /opt/anaconda/envs/Python3 : /opt/anaconda/envs/Python3/include/python3.6m : /opt/anaconda/envs/Python3/lib ;
   ``` 
   
   If you are not using Faculty, if you are
   targetting another version of Python, or if your Python installation is
   located elsewhere, you will need to modify this step accordingly.  The
   essential point is to make sure that the fourth field points to the directory
   containing the Python C header files (in particular, it should contain the
   file `Python.h`).  Now compile the Boost Python library with

   ```
   ./b2 install --with-python stage
   ```
   
   The Boost Python and Numpy shared libraries will
   be installed in `$BOOST_ROOT/stage/lib`.

Let's also set a number of environment variables to make things tidier.

* `BOOST_ROOT`: we have already defined this variable.
  It should point to the directory containing the Boost C++ headers.

* `BOOST_LIB`: this variable should point to the directory containing the shared
  Boost Python and Boost Numpy libraries. To define it, run 
  
  ```
  export BOOST_LIB=$BOOST_ROOT/stage/lib
  ```

* `PYTHON_INCLUDE`: this variable should point to the directory containing the
  Python header files. On Faculty, this would currently be set as follows:
  
  ```
  export PYTHON_INCLUDE=/opt/anaconda/envs/Python3/include/python3.6m
  ```

* `NUMPY_INCLUDE`: this variable should point to the directory containing the
  numpy header files.  On Faculty, this would currently be set as follows:

  ```
  export NUMPY_INCLUDE=/opt/anaconda/envs/Python3/lib/python3.6/site-packages/numpy/core/include
  ```

Additionally, `$BOOST_LIB` should be part of your `$LD_LIBRARY_PATH` environment
variable. This is to make sure that the Python interpreter is able to find the
shared Boost Python library we just compiled. Run the following
command to make sure that `LD_LIBRARY_PATH` is correctly set:

``` 
export LD_LIBRARY_PATH=$BOOST_LIB:$LD_LIBRARY_PATH
```

Note that this is the only environment variable that is required to be set
after installation. On Faculty, you may want to create a file `envs.sh` in 
`/etc/faculty_environment.d` defining the variables above. Then restart Jupyter with 
```
sudo sv stop jupyter && sudo sv start jupyter
```
The IPython kernels run by your Jupyter server will now have the variables correctly set

Let's give Boost Python a go! Here is a simple example defining our function `f` as part of a module `add_integers_boost`.

In [ ]:
!cat add_integers_boost.cpp

In [ ]:
!g++ add_integers_boost.cpp \
     -fPIC \
     -shared \
     -I $BOOST_ROOT \
     -I $PYTHON_INCLUDE \
     -L $BOOST_LIB \
     -lboost_python36 \
     -Wno-deprecated-declarations \
     -o add_integers_boost.so

A word on the options:
* `-fPIC` tells the compiler to generate position independent code, meaning that the code does not rely on where it is located in memory to be run. For example, jumps will be relative rather than absolute. This is required for shared libraries.
* `-shared` tells the compiler to create a shared library.
* `-I $BOOST_ROOT`, `-I $PYTHON_INCLUDE` and `-I $BOOST_LIB` give the compiler additional directories to look for header files
* `-lboost_python36` tells the compiler to dynamically link the compiled library against the Boost Python library (for version 3.6)
* `-Wno-deprecated-declarations` removes some warnings coming from the Boost headers (this can be omitted)
* `-o add_integers_boost.so` specifies the output file name of the library.

In [ ]:
from add_integers_boost import f

In [ ]:
f(100)

Here is an example of a C++ class exposed as a Python class:

In [ ]:
!cat accumulator_boost.cpp

In [ ]:
!g++ accumulator_boost.cpp \
     -fPIC \
     -shared \
     -I $BOOST_ROOT \
     -I $PYTHON_INCLUDE \
     -L $BOOST_LIB \
     -lboost_python36 \
     -Wno-deprecated-declarations \
     -o accumulator_boost.so

In [ ]:
from accumulator_boost import Accumulator

In [ ]:
acc = Accumulator()

In [ ]:
acc.total

In [ ]:
acc.add(5)

In [ ]:
acc.total

You typically will only want to implement the critical code paths in C++ - and hence only part of your class in C++. Therefore you might want to wrap the Boost-generated class in another class. For example:

In [ ]:
from accumulator_boost import Accumulator as CoreAccumulator


class Accumulator(CoreAccumulator):
    def __init__(self):
        self._core = CoreAccumulator()

    def add(self, increment):
        self._core.add(increment)

    @property
    def total(self):
        return self._core.total

    def help(self):
        print("Use this class to accumulate numbers.")

    def __repr__(self):
        return "Accumulator: total = {}".format(self.total)

In [ ]:
acc = Accumulator()

In [ ]:
acc.add(5)

In [ ]:
print(acc)

Although quite bare, the Boost Python documentation does have a helpful tutorial which you might want to check out - see the Boost documentation. In particular, it will show you how to interact with Numpy.

### Can I code a C extension from scratch?

Yes. Good luck :)

## Parallelisation

Let's talk about parallelisation... 

### System processes and threads

In computing a *process* is an instance of program being executed. At any one time, your computer is running tens of processes, if not hundreds. Here's how to get the number of running user processes on Linux:

In [ ]:
!ps -A --no-headers | wc -l

Note that you are very likely to be running (many) more processes than you have CPU cores! How is this possible? In effect, only one process is ever being run at a time on a machine with a single core. Processes *share CPU time*: the CPU constantly switches between them, and this gives the illusion that processes are being run concurrently. The part of your operating system responsible for orchestrating this is called the *scheduler*. It divides time into slices, and decides which process should be run during each slice. Of course, when you have more than one core, the scheduler will use them all: processes will be scheduled to run on different cores.

 A process can run one or more *threads*: these are sequences of execution sharing the same address space (so that communication between threads is much faster than communication between processes). So your system runs many processes, and each process can run many threads. Again, there is no relationship between number of cores and number of threads, but having more cores means that threads can run concurrently. To speed up the performance of you programmes, you might therefore want to make them *multithreaded*. This typically will make your programmes run faster on multi-core machines, but not always! It all depends on whether the bottleneck in the execution of your programme is the CPU, or something else. To give an example, if your programme is transcoding a video, then the bottleneck is the CPU - we say that it is *CPU-bound*. Video transcoders are typically multithreaded, and this significantly improves performance on multi-core machines. If your programme is downloading data from the web, then its performance is probably limited by the network bandwidth available to you - it is *I/O-bound*. In this context, the CPU is probably spending most of its time idling and waiting for packets to come from the source. Dividing the downloading task into two threads would probably not speed things up. 

### The `threading` package

You can create threads in Python with the `threading` package. Run the following cells on a machine with at least two cores.

In [ ]:
import threading

In [ ]:
from add_integers_python import f

In [ ]:
def single_threaded():
    return f(100_000_000) + f(100_000_000)

In [ ]:
timefunc(single_threaded)

In [ ]:
def f_adapted(index, results, *args, **kwargs):
    results[index] = f(*args, **kwargs)

def multi_threaded():
    results = {}
    t0 = threading.Thread(target=f_adapted, args=(0, results, 100_000_000))
    t1 = threading.Thread(target=f_adapted, args=(1, results, 100_000_000))
    t0.start()
    t1.start()
    t0.join()
    t1.join()
    return results[0] + results[1]

In [ ]:
timefunc(multi_threaded)

That was not much faster than the single-threaded version... in fact, it may even be a bit slower! Here is the problem:
Python threads *cannot be run concurrently*. This is a feature of the Python interpreter - the same programme written in C
would indeed be twice as fast as the single-threaded version. The Python interpreter has a so-called *Global Intepreter Lock (GIL)* - when a thread is being run by the interpreter, the GIL is set, which prevents other threads from running. In effect, the Python interpreter works like a single-core virtual machine. More on this here: https://www.dabeaz.com/python/UnderstandingGIL.pdf

This does not mean that using threads in Python is not a good idea: they can be very useful if your programme has to run independent streams of code concurrently: for example to handle multiple connections to a server at the same time. But you shouldn't expect performance improvements on CPU-bound tasks from using Python threads.

### The `multiprocessing` package

Python does have a way of making use of multicore machines: the `multiprocessing` package.

In [ ]:
import multiprocessing

In [ ]:
def multi_processed():
    manager = multiprocessing.Manager()
    results = manager.dict()
    t0 = multiprocessing.Process(
        target=f_adapted, args=(0, results, 100_000_000)
    )
    t1 = multiprocessing.Process(
        target=f_adapted, args=(1, results, 100_000_000)
    )
    t0.start()
    t1.start()
    t0.join()
    t1.join()
    return results[0] + results[1]

In [ ]:
timefunc(multi_processed)

This *does* improve performance on multicore machines. As the name indicates, the way that this module works is by creating independent *processes* rather than *threads*: a distinct Python intepreter (and hence a distinct system process) will be run for each "process" that you create in this way. This means, in particular,  that these processes do not share the same address space. This works well only when a small amount of inter-process communication is required. Besides, running a new Python interpreter for each process has its own inconveniences and overheads.

Naturally, if you choose to write a C/C++ extension (see above) then you can easily bypass the GIL mechanism, and you won't be limited by the `threading` or `multiprocessing` modules. And how does the (in?)famous `joblib` module come into the picture? Joblib will either use `threading` or `multiprocessing` depending on which backend you specify - and hence suffers from the same limitations as these modules.

## Acknowledgements

Many thanks to Scott Stevenson, Ben Green and Markus Kunesch for reviewing an earlier version of this notebook.